In [1]:
import requests, pymongo, json, time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup

In [2]:
client = pymongo.MongoClient()

In [3]:
def init_mongo_client():
    # Initiate Mongo client
    client = pymongo.MongoClient()

    # Access database created for these articles
    db = client.nyt_apr_4
    
    # Access collection created for these articles
    coll = db.NYT_Articles

    # Access articles collection in db and return collection pointer.
    return db.articles


def call_api(url, payload, p=0):
    # Add the 'page' parameter to the payload.
    payload['page'] = p

    # Get the requested url. Error handling for bad requests should be done in
    # the calling function.
    return requests.get(url, params=payload)


def get_response(r):
    # Use json.loads to read the response text
    raw = json.loads(r.text)

    # Return the meta (hits, etc.) and docs (containing urls'n'stuff) back
    return raw['response']['meta'], raw['response']['docs']


def get_soup(url):
    # Header to be passed in to NYT when scraping article text.
    agent  = 'DataWrangling/1.1 (http://zipfianacademy.com; '
    agent += 'class@zipfianacademy.com)'
    headers = {'user_agent': agent}

    # Wrap in a try-except to prevent a maxTry connection error from erroring
    # out the program. Return None if there are any issues.
    try:
        r = requests.get(url, headers=headers)
    except:
        return None

    # Just in case there was a normal error returned. Pass back None.
    if r.status_code != 200: return None

    # Otherwise return a soupified object containing the url text encoded in
    # utf-8. Will toss back errors on some pages without the encoding in place.
    return BeautifulSoup(r.text.encode('utf-8'),"lxml")


def get_body_text(docs):

    # Grab the url from each document, if it exists, then scrape each url for
    # its body text. If we get any errors along the way, continue on to the
    # next document / url to be scraped.
    result = []
    for d in docs:

        # Make a copy of the doc's dictionary
        doc = d.copy()

        # If there's no url (not sure why this happens sometimes) then ditch it
        if not doc['web_url']:
            continue

        # Scrape the doc's url, return a soup object with the url's text.
        soup = get_soup(doc['web_url'])
        if not soup:
            continue

        # Find all of the paragraphs with the correct class.
        # This class tag is specific to NYT articles.
        body = soup.find_all('p', class_= "story-body-text story-content")
        if not body:
            continue

        # Join the resulting body paragraphs' text (returned in a list).
        doc['body'] = '\n'.join([x.get_text() for x in body])

        print (doc['web_url'])
        result.append(doc)

    return result


def remove_previously_scraped(coll, docs):
    # Check to see if the mongo collection already contains the docs returned
    # from NYT. Return back a list of the ones that aren't in the collection to
    # be scraped.
    new_docs = []
    for doc in docs:
        # Check fo the document id in mongo. If it finds none, append to
        # new_docs
        cursor = articles.find({'_id': doc['_id']}).limit(1)
        if not cursor.count() > 0:
            new_docs.append(doc)

    if new_docs == []:
        return None

    return new_docs


def get_end_date(dt):
    # String-ify the datetime object to YYYMMDD, which the NYT likes.
    yr   = str(dt.year)
    mon = '0' * (2 - len(str(dt.month))) + str(dt.month)
    day = '0' * (2 - len(str(dt.day))) + str(dt.day)
    return yr + mon + day


def scrape_articles(coll, last_date):
    page = 0
    while page <= 199:
        print ('Page:', page)

        # Request all of the newest articles matching the search term
        payload  = {'sort': 'newest',
                    'end_date': get_end_date(last_date),
                    'api-key': API_KEY
                   }

        # Call the API with BaseURL + params and page
        r = call_api(NYT_URL, payload, page)

        # Increment the page before we encounter any potential errors
        page += 1

        # Check to see if the metadata didn't come back. USUALLY happens if
        # page > 100. When it does, reset the whole thing, roll the date back
        # one day, sleep for a couple seconds, then keep going.
        if r.status_code != 200:
            page = 0
            last_date += relativedelta(days=-1)
            print ('End Date:', get_end_date(last_date))
            print (r.status_code )
            time.sleep(2)
            continue
            
        # Get the meta data & documents from the request
        meta, docs = get_response(r)

        # Check if docs are already in the database
        new_docs = remove_previously_scraped(coll, docs)
        if not new_docs: continue

        # Grab only the docs that have these tags
        docs_with_body = get_body_text(new_docs)

        for doc in docs_with_body:
            try:
                # Insert each doc into Mongo
                coll.insert_one(doc)
            except:
                # If there's any error writing it in the db, just move along.
                continue

In [4]:
# Url for NYT dev api
NYT_URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
api_key_path = "/home/sathyanarayanan/Desktop/NLP project/FAKE NEWS DETECTION/NYT_API_Key.txt"

In [5]:
with open(api_key_path, 'r') as handle:
    API_KEY = handle.read()

In [6]:
# Initialize db & collection
articles = init_mongo_client()

In [7]:
# Set the initial end date (scraper starts at this date and moves back in
# time sequentially)
last_date = datetime.now() + relativedelta(days=-2)

In [8]:
# Pass the database collection and initial end date into main function
scrape_articles(articles, last_date)

Page: 0
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-xgr-connecticut-chief-justice.html
https://www.nytimes.com/reuters/2018/03/27/world/asia/27reuters-china-northkorea.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-hawaii-obama-statue.html
https://www.nytimes.com/reuters/2018/03/27/world/asia/27reuters-northkorea-missiles-china-yonhap.html
https://www.nytimes.com/2018/03/27/us/cyberattack-atlanta-ransomware.html
https://www.nytimes.com/reuters/2018/03/27/business/27reuters-japan-ageing-robots.html
https://www.nytimes.com/reuters/2018/03/27/sports/soccer/27reuters-soccer-friendly-esp-arg-messi.html
https://www.nytimes.com/2018/03/27/world/asia/kim-jong-un-china-north-korea.html
https://www.nytimes.com/2018/03/27/us/politics/trump-giuseppe-cecchi-donors.html
https://www.nytimes.com/reuters/2018/03/27/sports/golf/27reuters-golf-women-inspiration.html
Page: 1
https://www.nytimes.com/aponline/2018/03/27/us/ap-ca-california-death-penalty-the-latest.html
https://www.nytime

https://www.nytimes.com/reuters/2018/03/27/sports/soccer/27reuters-soccer-friendly-eng-ita.html
https://www.nytimes.com/reuters/2018/03/27/world/middleeast/27reuters-israel-netanyahu-condition.html
https://www.nytimes.com/reuters/2018/03/27/us/27reuters-alabama-execution.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-financial-markets-box.html
Page: 10
https://www.nytimes.com/reuters/2018/03/27/world/africa/27reuters-congo-politics.html
https://www.nytimes.com/reuters/2018/03/27/arts/27reuters-people-tigerwoods.html
https://www.nytimes.com/2018/03/27/business/tech-stocks-tesla-nvidia.html
https://www.nytimes.com/2018/03/27/science/nasa-webb-telescope.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-student-gun-protests-cuba-flag.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-mormon-church-sexual-abuse.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-child-porn-video.html
https://www.nytimes.com/reuters/2018/03/27/sports/soccer/27reuters-soccer-frien

https://www.nytimes.com/aponline/2018/03/27/world/middleeast/ap-ml-egypt-election-turnout.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-turkey-us-corruption-trial.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-facebook-lawsuit.html
https://www.nytimes.com/aponline/2018/03/27/sports/baseball/ap-bbo-increasingly-elusive-3.html
https://www.nytimes.com/2018/03/27/us/stephon-clark-police-killing.html
https://www.nytimes.com/2018/03/27/us/politics/census-citizenship-question.html
https://www.nytimes.com/2018/03/27/style/angels-in-america-broadway-premiere.html
https://www.nytimes.com/aponline/2018/03/27/sports/basketball/ap-bkn-76ers-here-they-come.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-orlando-shooters-wife.html
Page: 21
https://www.nytimes.com/reuters/2018/03/27/world/middleeast/27reuters-usa-immigration-travelban.html
https://www.nytimes.com/2018/03/27/fashion/weddings/real-life-can-be-raw-in-marriage-and-death.html
https://www.nytimes.com/reuters

https://www.nytimes.com/aponline/2018/03/27/us/ap-us-fertility-clinic-failures-the-latest.html
https://www.nytimes.com/reuters/2018/03/27/world/africa/27reuters-libya-security-zintan.html
https://www.nytimes.com/2018/03/27/sports/baseball/josh-gibson-buck-leonard-negro-leagues.html
https://www.nytimes.com/reuters/2018/03/27/business/27reuters-telecomitalia-board-probe.html
https://www.nytimes.com/2018/03/27/business/tanzina-vega-wnyc-takeaway.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-fatal-dog-attack.html
https://www.nytimes.com/2018/03/27/theater/kirill-serebrennikov-russia-germany.html
https://www.nytimes.com/2018/03/27/us/william-strampel-michigan-state.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-california-gray-whales.html
https://www.nytimes.com/aponline/2018/03/27/us/ap-us-school-threat-arrest.html
Page: 31
https://www.nytimes.com/aponline/2018/03/27/us/politics/ap-us-trump-dinner.html
https://www.nytimes.com/reuters/2018/03/27/world/europe/27reuters

https://www.nytimes.com/aponline/2018/03/23/us/ap-us-actress-fatal-accident-the-latest.html
https://www.nytimes.com/aponline/2018/03/23/us/ap-us-courthouse-scandal-felon-hired.html
https://www.nytimes.com/2018/03/23/health/right-to-try-drugs-fda.html
https://www.nytimes.com/reuters/2018/03/23/business/23reuters-american-airlines-boeing-jet.html
https://www.nytimes.com/2018/03/23/technology/trump-china-tariffs-tech-cold-war.html
https://www.nytimes.com/aponline/2018/03/23/sports/baseball/ap-bba-twins-sano-no-discipline.html
https://www.nytimes.com/reuters/2018/03/23/business/23reuters-stada-arzneimitt-listing-generalstandard.html
https://www.nytimes.com/2018/03/23/us/politics/jeff-sessions-gun-violence-bump-stocks.html
Page: 8
https://www.nytimes.com/reuters/2018/03/23/world/americas/23reuters-brazil-corruption-church.html
https://www.nytimes.com/reuters/2018/03/23/us/politics/23reuters-usa-guns-bumpstocks.html
https://www.nytimes.com/aponline/2018/03/23/us/ap-us-fatal-fire-movie-set-th

...

https://www.nytimes.com/2015/10/08/t-magazine/runway-songs-new-york-london-milan-paris-fashion-week.html
Page: 129
https://www.nytimes.com/2015/10/08/movies/new-york-film-festival-five-questions-for-michael-almereyda.html
https://www.nytimes.com/2015/10/08/universal/es/que-beneficios-trae-el-consumo-de-te-esto-es-lo-que-dice-la-evidencia.html
https://www.nytimes.com/2015/10/08/t-magazine/best-jewelry-paris-fashion-week.html
Page: 130
https://www.nytimes.com/2015/10/08/nytnow/latest-news-syria-kevin-mccarthy-chicago-cubs.html
https://www.nytimes.com/2015/10/08/pageoneplus/corrections-october-8-2015.html
https://www.nytimes.com/2015/10/08/business/media/vimeo-expands-its-lineup-of-original-offerings.html
https://www.nytimes.com/2015/10/08/pageoneplus/quotation-of-the-day-for-thursday-october-8-2015.html
https://www.nytimes.com/2015/10/08/travel/hotel-and-food-news-casual-dining-in-paris-puglia-by-bike.html
Page: 131
https://www.nytimes.com/2015/10/08/business/amazon-challenges-etsy-with-

https://www.nytimes.com/2015/10/08/arts/music/review-schubert-via-david-fray-at-the-park-avenue-armory.html
Page: 144
https://www.nytimes.com/2015/10/08/world/middleeast/violence-spreads-in-israel-despite-security-crackdown.html
https://www.nytimes.com/2015/10/08/world/europe/hungarian-fashion-photographer-defends-migrant-chic-spread.html
https://www.nytimes.com/2015/10/08/arts/television/review-amazons-red-oaks-aims-for-80s-affection-not-satire.html
https://www.nytimes.com/2015/10/08/us/testimony-on-us-nutrition-guidelines-reflects-a-complex-debate.html
Page: 145
https://www.nytimes.com/2015/10/08/arts/dance/kim-brandstrup-brings-jeux-and-follows-a-different-path-to-city-ballet.html
https://www.nytimes.com/2015/10/08/arts/dance/fall-for-dance-at-city-center-the-old-man-shows-how-it-is-done.html
https://www.nytimes.com/2015/10/08/us/some-see-refusals-to-serve-policemen-as-a-backlash.html
https://www.nytimes.com/2015/10/08/arts/music/review-dan-truemans-nostalgic-synchronic-takes-cage-i

https://www.nytimes.com/2015/10/08/technology/personaltech/getting-directions-while-pedaling.html
https://www.nytimes.com/2015/10/08/business/ge-to-spin-off-new-energy-technologies-in-one-company.html
Page: 160
https://www.nytimes.com/2015/10/08/upshot/joe-biden-no-money-weak-polls-but-still-clintons-toughest-rival.html
https://www.nytimes.com/2015/10/08/arts/international/review-nell-gwynn-and-medea-in-london.html
https://www.nytimes.com/2015/10/08/business/media/pandora-buys-ticketfly-a-competitor-to-ticketmaster.html
https://www.nytimes.com/2015/10/08/world/asia/malaysia-sultans-najib-razak-1mdb-corruption.html
Page: 161
https://www.nytimes.com/2015/10/11/realestate/ardsley-new-york-village.html
Page: 162
https://www.nytimes.com/2015/10/08/sports/golf/golf-is-coming-on-strong-in-asia-presidents-cup.html
https://www.nytimes.com/2015/10/11/travel/restaurants-chile-santiago.html
https://www.nytimes.com/2015/10/08/theater/womens-voices-theater-festival-in-washington-is-an-energizing-sho

https://www.nytimes.com/2015/10/07/sports/baseball/celebration-plans-turn-to-restraint-followed-quickly-by-airplane-trip.html
https://www.nytimes.com/2015/10/07/sports/basketball/american-on-brazilian-team-relishes-facing-knicks.html
https://www.nytimes.com/2015/10/07/sports/football/nfl-roundup.html
https://www.nytimes.com/2015/10/07/world/asia/parramatta-australia-shooting-terrorism-cited.html
https://www.nytimes.com/2015/10/07/business/teamsters-pension-fund-warns-400000-of-cuts.html
https://www.nytimes.com/2015/10/07/world/middleeast/obama-turns-to-crowdfunding-to-aid-fleeing-syrians.html
https://www.nytimes.com/2015/10/07/business/dealbook/dewey-leboeuf-jurors-tell-judge-they-are-stuck-after-13-days-of-deliberation.html
https://www.nytimes.com/2015/10/07/business/uaw-warns-fiat-chrysler-to-make-deal-or-face-strike.html
https://www.nytimes.com/2015/10/07/sports/ncaabasketball/chris-mullin-hires-mitch-richmond-as-st-johns-assistant.html
Page: 180
https://www.nytimes.com/2015/10/07/b

https://www.nytimes.com/2015/10/08/fashion/nail-art-streamlining.html
https://www.nytimes.com/2015/10/07/world/americas/honduras-el-salvador-guatemala-mexico-us-child-migrants.html
Page: 191
https://www.nytimes.com/2015/10/07/technology/the-hardware-side-of-microsoft-unveils-a-pile-of-new-devices.html
https://www.nytimes.com/2015/10/07/us/politics/hillary-clintons-proposed-changes-to-health-law-zero-in-on-affordability.html
Page: 192
https://www.nytimes.com/2015/10/07/nyregion/man-is-killed-and-2-are-injured-in-east-williamsburg-shooting.html
https://www.nytimes.com/2015/10/07/upshot/the-big-winners-in-fantasy-football-and-the-rest-of-us.html
https://www.nytimes.com/2015/10/07/business/dealbook/sabmillers-trading-statement-may-alter-potential-budweiser-bid.html
https://www.nytimes.com/2015/10/07/world/europe/westminster-vip-pedophile-claims.html
https://www.nytimes.com/2015/10/07/sports/golf/presidents-cup-could-feature-golfs-no-1-vs-no-2.html
Page: 193
https://www.nytimes.com/2015/10/